In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import scipy as sp
from scipy import stats

from sklearn import preprocessing
import xgboost as xgb
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score

import gc
gc.enable()

print("Ready")

Ready


In [2]:
# Utility functions

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
df_trans = pd.read_csv('../input/train_transaction.csv')
df_test_trans = pd.read_csv('../input/test_transaction.csv')

df_id = pd.read_csv('../input/train_identity.csv')
df_test_id = pd.read_csv('../input/test_identity.csv')

sample_submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')

df_train = df_trans.merge(df_id, how='left', left_index=True, right_index=True, on='TransactionID')
df_test = df_test_trans.merge(df_test_id, how='left', left_index=True, right_index=True, on='TransactionID')

print(df_train.shape)
print(df_test.shape)

# y_train = df_train['isFraud'].copy()
del df_trans, df_id, df_test_trans, df_test_id

# reduce memory usage (leads to overflow)
# df_train = reduce_mem_usage(df_train)
# df_test = reduce_mem_usage(df_test)

(590540, 434)
(506691, 433)


# Feature Engineering

In [11]:
df_train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [12]:
df_train = df_train.drop('index', axis=1)
df_test = df_test.drop('index', axis=1)

ValueError: labels ['index'] not contained in axis

## Mapping Emails

In [13]:
# copy pastaed for now
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 
          'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft',
          'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 
          'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other',
          'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo',
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft',
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other',
          'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}

us_emails = ['gmail', 'net', 'edu']

# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654
for c in ['P_emaildomain', 'R_emaildomain']:
    df_train[c + '_bin'] = df_train[c].map(emails)
    df_test[c + '_bin'] = df_test[c].map(emails)
    
    df_train[c + '_suffix'] = df_train[c].map(lambda x: str(x).split('.')[-1])
    df_test[c + '_suffix'] = df_test[c].map(lambda x: str(x).split('.')[-1])
    
    df_train[c + '_suffix'] = df_train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    df_test[c + '_suffix'] = df_test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')



## Transaction Amount

In [14]:
df_train['Trans_min_mean'] = df_train['TransactionAmt'] - df_train['TransactionAmt'].mean()
df_train['Trans_min_std'] = df_train['Trans_min_mean'] / df_train['TransactionAmt'].std()
df_test['Trans_min_mean'] = df_test['TransactionAmt'] - df_test['TransactionAmt'].mean()
df_test['Trans_min_std'] = df_test['Trans_min_mean'] / df_test['TransactionAmt'].std()

df_train['TransactionAmt_to_mean_card1'] = df_train['TransactionAmt'] / df_train.groupby(['card1'])['TransactionAmt'].transform('mean')
df_train['TransactionAmt_to_mean_card4'] = df_train['TransactionAmt'] / df_train.groupby(['card4'])['TransactionAmt'].transform('mean')
df_train['TransactionAmt_to_std_card1'] = df_train['TransactionAmt'] / df_train.groupby(['card1'])['TransactionAmt'].transform('std')
df_train['TransactionAmt_to_std_card4'] = df_train['TransactionAmt'] / df_train.groupby(['card4'])['TransactionAmt'].transform('std')

df_test['TransactionAmt_to_mean_card1'] = df_test['TransactionAmt'] / df_test.groupby(['card1'])['TransactionAmt'].transform('mean')
df_test['TransactionAmt_to_mean_card4'] = df_test['TransactionAmt'] / df_test.groupby(['card4'])['TransactionAmt'].transform('mean')
df_test['TransactionAmt_to_std_card1'] = df_test['TransactionAmt'] / df_test.groupby(['card1'])['TransactionAmt'].transform('std')
df_test['TransactionAmt_to_std_card4'] = df_test['TransactionAmt'] / df_test.groupby(['card4'])['TransactionAmt'].transform('std')

df_train['TransactionAmt_log'] = np.log(df_train['TransactionAmt'])
df_test['TransactionAmt_log'] = np.log(df_test['TransactionAmt'])

df_train['TransactionAmt_cents'] = df_train['TransactionAmt'] % 1
df_test['TransactionAmt_cents'] = df_test['TransactionAmt'] % 1



MemoryError: 

# Encode Categorical Features

In [7]:
for f in df_train.drop('isFraud', axis=1).columns:
    if df_train[f].dtype=='object' or df_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_train[f].values) + list(df_test[f].values))
        df_train[f] = lbl.transform(list(df_train[f].values))
        df_test[f] = lbl.transform(list(df_test[f].values))   

## PCA V Features

In [8]:
# TODO encode categorical features

# current approach does PCA on training and test data which is data snooping
# for now just do PCA on the training set and apply that transformation on
# the test test
# TODO: PCA on every CV fold
# source: https://stats.stackexchange.com/questions/55718/pca-and-the-train-test-split
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA

def PCA_change(df, cols, n_components, prefix="PCA_", rand_seed=4):
    # returns df (cols replaces with PCA), pca (to transform test data)
    pca = PCA(n_components=n_components, random_state=rand_seed)
    pca.fit(df[cols])
    principalComponents = pca.transform(df[cols])

    print("explained variance", pca.explained_variance_)

    principalDf = pd.DataFrame(principalComponents)

    df.drop(cols, axis=1, inplace=True)

    principalDf.rename(columns=lambda x: str(prefix)+str(x), inplace=True)

    df = pd.concat([df, principalDf], axis=1)

    return df, pca

def PCA_transform(df, cols, pca, prefix="PCA_"):
    principalComponents = pca.transform(df[cols])
    principalDf = pd.DataFrame(principalComponents)

    df.drop(cols, axis=1, inplace=True)

    principalDf.rename(columns=lambda x: str(prefix)+str(x), inplace=True)

    df = pd.concat([df, principalDf], axis=1)

    return df    

def fillna(df, cols):
    for col in cols:
        df[col] = df[col].fillna((df[col].min() - 2))
        df[col] = (minmax_scale(df[col], feature_range=(0,1)))

In [9]:
mas_v = df_train.columns[55:394]

# prepare data for PCA
df = df_train
fillna(df, mas_v)

df, pca_train = PCA_change(df, mas_v, prefix='PCA_V_', n_components=30)
df = reduce_mem_usage(df)

explained variance [  1.33501052e+00   4.17885342e-01   2.17001711e-01   9.61400981e-02
   5.61497797e-02   4.80469204e-02   3.77956377e-02   3.04519753e-02
   2.66785618e-02   1.63133780e-02   7.25608269e-03   5.55054641e-03
   4.76129330e-03   4.44079966e-03   3.95344590e-03   3.47549388e-03
   3.31677665e-03   3.10927176e-03   2.96483588e-03   2.79745706e-03
   2.53528055e-03   2.37490214e-03   2.11353364e-03   1.96469996e-03
   1.87787500e-03   1.65282338e-03   1.56319475e-03   1.49009459e-03
   1.38632359e-03   1.31973239e-03]
Mem. usage decreased to 142.49 Mb (76.9% reduction)


In [9]:
tot_explained_var = sum([1.33501052e+00, 4.17885342e-01, 2.17001711e-01, 9.61400981e-02,
 5.61497797e-02, 4.80469204e-02, 3.77956377e-02, 3.04519753e-02,
 2.66785618e-02, 1.63133780e-02, 7.25608269e-03, 5.55054641e-03,
 4.76129330e-03, 4.44079966e-03, 3.95344590e-03, 3.47549388e-03,
 3.31677665e-03, 3.10927176e-03, 2.96483588e-03, 2.79745706e-03,
 2.53528055e-03, 2.37490214e-03, 2.11353364e-03, 1.96469996e-03,
 1.87787500e-03, 1.65282338e-03, 1.56319475e-03, 1.49009459e-03,
 1.38632359e-03, 1.31973239e-03])

print(f"Explained variance: {tot_explained_var} (Holy shit the explained variance is very low)")

Explained variance: 2.34137838718 (Holy shit the explained variance is very low)


In [10]:
# project test data as well
fillna(df_test, mas_v)
df_test = PCA_transform(df_test, mas_v, pca_train)

In [11]:
df_test.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,PCA_20,PCA_21,PCA_22,PCA_23,PCA_24,PCA_25,PCA_26,PCA_27,PCA_28,PCA_29
0,3663549,18403224,31.95,4,10409,111.0,150.0,4,226.0,2,...,-0.151680,0.041224,0.178498,0.044082,0.011070,-0.085637,-0.055372,-0.070176,0.141670,0.169869
1,3663550,18403263,49.00,4,4272,111.0,150.0,4,226.0,2,...,-0.175929,-0.023710,0.163335,0.077521,0.024231,0.077039,0.039543,-0.019159,0.051953,0.188614
2,3663551,18403310,171.00,4,4476,574.0,150.0,4,226.0,2,...,-0.194145,-0.075302,0.178145,0.078438,0.026147,0.062222,0.021844,-0.007461,0.051529,0.216012
3,3663552,18403310,284.95,4,10989,360.0,150.0,4,166.0,2,...,-0.131444,0.088380,0.165277,0.058273,0.020537,0.033675,0.053800,-0.022234,0.061329,0.238952
4,3663553,18403317,67.95,4,18018,452.0,150.0,2,117.0,2,...,-0.138103,0.019570,0.150636,0.075586,0.025711,0.044335,0.046026,-0.026404,0.039927,0.174710


## Set X and y

In [49]:
df_train = df
df_train_TransactionDT_sorted = df_train.sort_values('TransactionDT')
X_train = df_train_TransactionDT_sorted.drop(['isFraud', 'TransactionDT', 'TransactionID'], axis=1)
y_train = df_train_TransactionDT_sorted['isFraud'].astype(bool)

X_test = df_test.sort_values('TransactionDT').drop(['TransactionDT', 'TransactionID'], 
                                                   axis=1)
del df_train
df_test = df_test[["TransactionDT"]] # why did the original notebook use this

ValueError: labels ['TransactionID'] not contained in axis

In [24]:
df_test = df_test[["TransactionDT"]]

In [13]:
print("X_train", len(X_train.columns))
print("X_test", len(X_test.columns))

X_train 134
X_test 134


# Defining the HyperOpt function with parameters space and model

In [39]:
## Hyperopt modules
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from functools import partial

from sklearn.model_selection import KFold,TimeSeriesSplit
from sklearn.metrics import roc_auc_score
from xgboost import plot_importance
from sklearn.metrics import make_scorer

import time

global_x_vl, global_y_vl = None, None
def objective(params):
    # debugging
    global global_x_vl, global_y_vl, global_scorer, global_clf
    
    time1 = time.time()
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'subsample': "{:.2f}".format(params['subsample']),
        'reg_alpha': "{:.3f}".format(params['reg_alpha']),
        'reg_lambda': "{:.3f}".format(params['reg_lambda']),
        'learning_rate': "{:.3f}".format(params['learning_rate']),
        'num_leaves': '{:.3f}'.format(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'min_child_samples': '{:.3f}'.format(params['min_child_samples']),
        'feature_fraction': '{:.3f}'.format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction'])
    }

    print("\n############## New Run ################")
    print(f"params = {params}")
    FOLDS = 7
    count=1
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

    tss = TimeSeriesSplit(n_splits=FOLDS)
    y_preds = np.zeros(sample_submission.shape[0])
    y_oof = np.zeros(X_train.shape[0])
    score_mean = 0
    for tr_idx, val_idx in tss.split(X_train, y_train):
        clf = xgb.XGBClassifier(
            n_estimators=600, random_state=4, verbose=True, 
            tree_method='hist', 
            **params
        )

        X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
        y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        
        clf.fit(X_tr, y_tr)
        #y_pred_train = clf.predict_proba(X_vl)[:,1]
        #print(y_pred_train)
        
        # debugging
        global_x_vl = X_vl
        global_y_vl = y_vl
        global_scorer = make_scorer(roc_auc_score, needs_proba=True)
        global_clf = clf
        
        score = make_scorer(roc_auc_score, needs_proba=True)(clf, X_vl, y_vl)
        # plt.show()
        score_mean += score
        print(f'{count} CV - score: {round(score, 4)}')
        count += 1
    time2 = time.time() - time1
    print(f"Total Time Run: {round(time2 / 60,2)}")
    gc.collect()
    print(f'Mean ROC_AUC: {score_mean / FOLDS}')
    del X_tr, X_vl, y_tr, y_vl, clf, score
    return -(score_mean / FOLDS)


space = {
    # The maximum depth of a tree, same as GBM.
    # Used to control over-fitting as higher depth will allow model 
    # to learn relations very specific to a particular sample.
    # Should be tuned using CV.
    # Typical values: 3-10
    'max_depth': hp.quniform('max_depth', 7, 23, 1),
    
    # reg_alpha: L1 regularization term. L1 regularization encourages sparsity 
    # (meaning pulling weights to 0). It can be more useful when the objective
    # is logistic regression since you might need help with feature selection.
    'reg_alpha':  hp.uniform('reg_alpha', 0.01, 0.4),
    
    # reg_lambda: L2 regularization term. L2 encourages smaller weights, this
    # approach can be more useful in tree-models where zeroing 
    # features might not make much sense.
    'reg_lambda': hp.uniform('reg_lambda', 0.01, .4),
    
    # eta: Analogous to learning rate in GBM
    # Makes the model more robust by shrinking the weights on each step
    # Typical final values to be used: 0.01-0.2
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    
    # colsample_bytree: Similar to max_features in GBM. Denotes the 
    # fraction of columns to be randomly samples for each tree.
    # Typical values: 0.5-1
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, .9),
    
    # A node is split only when the resulting split gives a positive
    # reduction in the loss function. Gamma specifies the 
    # minimum loss reduction required to make a split.
    # Makes the algorithm conservative. The values can vary depending on the loss function and should be tuned.
    'gamma': hp.uniform('gamma', 0.01, .7),
    
    # more increases accuracy, but may lead to overfitting.
    # num_leaves: the number of leaf nodes to use. Having a large number 
    # of leaves will improve accuracy, but will also lead to overfitting.
    'num_leaves': hp.choice('num_leaves', list(range(20, 250, 10))),
    
    # specifies the minimum samples per leaf node.
    # the minimum number of samples (data) to group into a leaf. 
    # The parameter can greatly assist with overfitting: larger sample
    # sizes per leaf will reduce overfitting (but may lead to under-fitting).
    'min_child_samples': hp.choice('min_child_samples', list(range(100, 250, 10))),
    
    # subsample: represents a fraction of the rows (observations) to be 
    # considered when building each subtree. Tianqi Chen and Carlos Guestrin
    # in their paper A Scalable Tree Boosting System recommend 
    'subsample': hp.choice('subsample', [0.2, 0.4, 0.5, 0.6, 0.7, .8, .9]),
    
    # randomly select a fraction of the features.
    # feature_fraction: controls the subsampling of features used
    # for training (as opposed to subsampling the actual training data in 
    # the case of bagging). Smaller fractions reduce overfitting.
    'feature_fraction': hp.uniform('feature_fraction', 0.4, .8),
    
    # randomly bag or subsample training data.
    'bagging_fraction': hp.uniform('bagging_fraction', 0.4, .9)
    
    # bagging_fraction and bagging_freq: enables bagging (subsampling) 
    # of the training data. Both values need to be set for bagging to be used.
    # The frequency controls how often (iteration) bagging is used. Smaller
    # fractions and frequencies reduce overfitting.
}

In [29]:
## Running the optimizer

In [40]:
# Set algoritm parameters
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=27)

# Print best parameters
best_params = space_eval(space, best)

                                                    
############## New Run ################
params = {'max_depth': 9, 'gamma': '0.242', 'subsample': '0.40', 'reg_alpha': '0.043', 'reg_lambda': '0.286', 'learning_rate': '0.070', 'num_leaves': '60.000', 'colsample_bytree': '0.772', 'min_child_samples': '190.000', 'feature_fraction': '0.796', 'bagging_fraction': '0.405'}
  0%|          | 0/27 [00:49<?, ?it/s, best loss: ?]


ValueError: bad input shape (73817, 2)

In [46]:
global_y_vl.shape

(73817,)

In [47]:
global_x_vl.shape

(73817, 134)

In [48]:
df_train.shape

NameError: name 'df_train' is not defined